Notebook de récolte des data de 5 sources différentes : CSV, scrapping, base de données mySQL, base de donnée Mongo, API ainsi que le traitement de celles-ci pour les combiner

La note d'un film avec une note moyenne supérieure à 7.5 sera converti en 'bien', moins de 5 en 'mediocre', entre les 2 'moyen'

Initialisation des dépendances et connections

In [9]:
import pandas as pd
import requests
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from mysql_data import Data_mysql
from mongo_data import Data_mongo
from dotenv import load_dotenv
env_path = './bdd/.env'
load_dotenv(env_path)

True

**Récolte de data par csv**


In [39]:
try:
    # Lecture du csv
    df_csv = pd.read_csv('tmdb_5000_movies.csv')
    # affichage pour vérification
    print(df_csv.head())
except FileNotFoundError:
    print("Le fichier CSV n'a pas été trouvé.")
except pd.errors.EmptyDataError:
    print("Le fichier CSV est vide.")
except pd.errors.ParserError:
    print("Erreur lors de l'analyse du fichier CSV.")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [40]:
# Création d'une copie du Dataframe (pour ne pas modifier l'original) avec les colonnes que l'on veut
df_csv_copy = df_csv[['original_title', 'budget', 'revenue', 'runtime', 'vote_count']].copy()

# convertion de 'vote_average' en chaîne de caractères et création de la colonne 'evaluation'
df_csv_copy['evaluation'] = df_csv['vote_average'].astype(str)

# Supprime les lignes où 'runtime' est 0 ou NaN
df_csv_copy = df_csv_copy[df_csv_copy['runtime'].notna() & (df_csv_copy['runtime'] != 0)]

# Convertion de 'runtime' en chaîne de caractères, puis supprime tout ce qui suit le point décimal
df_csv_copy['runtime'] = df_csv_copy['runtime'].astype(str).str.split('.').str[0]

print(df_csv_copy.tail())


                 original_title  budget  revenue runtime  vote_count  \
4798                El Mariachi  220000  2040920      81         238   
4799                  Newlyweds    9000        0      85           5   
4800  Signed, Sealed, Delivered       0        0     120           6   
4801           Shanghai Calling       0        0      98           7   
4802          My Date with Drew       0        0      90          16   

     evaluation  
4798        6.6  
4799        5.9  
4800        7.0  
4801        5.7  
4802        6.3  


In [42]:
# Convertion de 'evaluation' en numérique (float)
df_csv_copy['evaluation'] = pd.to_numeric(df_csv_copy['evaluation'], errors='coerce')

# Appliquer les conditions pour la colonne 'evaluation'
df_csv_copy['evaluation'] = df_csv_copy['evaluation'].apply(
    lambda x: 'bien' if x > 7.5 else 'mediocre' if x < 5 else 'moyen'
)

In [43]:
df_csv_copy.head()

,original_title,budget,revenue,runtime,vote_count,evaluation
0,Avatar,237000000,2787965087,162,11800,moyen
1,Pirates of the Caribbean: At World's End,300000000,961000000,169,4500,moyen
2,Spectre,245000000,880674609,148,4466,moyen
3,The Dark Knight Rises,250000000,1084939099,165,9106,bien
4,John Carter,260000000,284139100,132,2124,moyen


**Récolte de data par bdd mySQL**

In [ ]:
try:
    data_instance_mysql = Data_mysql()

    # Appelle la méthode get_films_data_mysql via l'instance
    films_data_mysql = data_instance_mysql.get_films_data_mysql()

    df_mysql = pd.DataFrame(films_data_mysql)

    df_mysql.head()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")

   id                            original_title     budget     revenue  \
0   1                                    Avatar  237000000  2787965087   
1   2  Pirates of the Caribbean: At World's End  300000000   961000000   
2   3                                   Spectre  245000000   880674609   
3   4                     The Dark Knight Rises  250000000  1084939099   
4   5                               John Carter  260000000   284139100   

   runtime  vote_count evaluation  
0      162       11800      moyen  
1      169        4500      moyen  
2      148        4466      moyen  
3      165        9106       bien  
4      132        2124      moyen  


In [60]:
df_mysql = df_mysql.drop('id', axis=1)
df_mysql.head()

,original_title,budget,revenue,runtime,vote_count,evaluation
0,Avatar,237000000,2787965087,162,11800,moyen
1,Pirates of the Caribbean: At World's End,300000000,961000000,169,4500,moyen
2,Spectre,245000000,880674609,148,4466,moyen
3,The Dark Knight Rises,250000000,1084939099,165,9106,bien
4,John Carter,260000000,284139100,132,2124,moyen


**Récolte de data par bdd Mongo**

In [ ]:
try:
    data_instance_mongo = Data_mongo()

    films_data_mongo = data_instance_mongo.get_films_data_mongo()  

    df_mongo = pd.DataFrame(films_data_mongo)

    df_mongo.head()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")

                       original_title    budget    revenue  runtime  \
0                Insidious: Chapter 3  10000000  104303851       97   
1                     The Last Dragon  10000000   25754284      108   
2                   The Lawnmower Man  10000000   32101000      108   
3  Nick and Norah's Infinite Playlist   9000000   32973937       89   
4                               Dogma  10000000   30652890      130   

   vote_count evaluation  
0         983      moyen  
1          69      moyen  
2         197      moyen  
3         379      moyen  
4         823      moyen  


**Récolte de data par l'API de TMDB**

In [ ]:
# URL pour récupérer les films populaires
url_popular = "https://api.themoviedb.org/3/movie/popular?language=en-US&page=1"

headers = {
    "accept": "application/json",
    "Authorization": os.getenv("API_KEY")  
}

# Faire la requête pour récupérer les films populaires
try:
    response = requests.get(url_popular, headers=headers)
    response.raise_for_status()  
    response_data = response.json()  

    # Extraire les IDs des films (limite aux 10 premiers films)
    movie_ids = [movie['id'] for movie in response_data['results'][:10]]

except requests.exceptions.RequestException as e:
    print(f"Erreur lors de la requête des films populaires: {e}")
    movie_ids = []  # Initialise une liste vide si la requête échoue

# URL pour récupérer les data d'un film en fonction de son id
url_details = "https://api.themoviedb.org/3/movie/movie_id?language=en-US"

# Initialise une liste pour stocker les data des films
movies_list = []

# Boucle pour récupérer les data des films
for movie_id in movie_ids:
    try:
        response = requests.get(url_details.replace("movie_id", str(movie_id)), headers=headers)
        response.raise_for_status()  
        movie_data = response.json()
        
        movie_details = {
            'original_title': movie_data.get('original_title', None),
            'budget': movie_data.get('budget', None),
            'revenue': movie_data.get('revenue', None),
            'runtime': movie_data.get('runtime', None),
            'vote_count': movie_data.get('vote_count', None),
            'vote_average': movie_data.get('vote_average', None)
        }

        # Ajoute les data des films à la liste
        movies_list.append(movie_details)

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête pour le film ID {movie_id}: {e}")
    except ValueError as ve:
        print(f"Erreur de parsing pour le film ID {movie_id}: {ve}")

# Crée un DataFrame à partir de la liste de films
df_api = pd.DataFrame(movies_list)

print(df_api.head())


         original_title     budget     revenue  runtime  vote_count  \
0  Deadpool & Wolverine  200000000  1317173104      128        3160   
1              The Crow   50000000    13690814      111         424   
2          Inside Out 2  200000000  1682636477       97        3906   
3       It Ends with Us   25000000   334836337      131         562   
4       Despicable Me 4  100000000   948140610       94        1767   

   vote_average  
0         7.672  
1         5.404  
2         7.642  
3         6.880  
4         7.166  


In [51]:
df_api_copy = df_api
df_api_copy['evaluation'] = df_api['vote_average'].apply(
    lambda x: 'bien' if x > 7.5 else ('mediocre' if x < 5 else 'moyen')
)

df_api_copy.head()

,original_title,budget,revenue,runtime,vote_count,vote_average,evaluation
0,Deadpool & Wolverine,200000000,1317173104,128,3160,7.672,bien
1,The Crow,50000000,13690814,111,424,5.404,moyen
2,Inside Out 2,200000000,1682636477,97,3906,7.642,bien
3,It Ends with Us,25000000,334836337,131,562,6.880,moyen
4,Despicable Me 4,100000000,948140610,94,1767,7.166,moyen


In [56]:
df_api_copy = df_api_copy.drop('vote_average', axis=1)
df_api_copy.head()

,original_title,budget,revenue,runtime,vote_count,evaluation
0,Deadpool & Wolverine,200000000,1317173104,128,3160,bien
1,The Crow,50000000,13690814,111,424,moyen
2,Inside Out 2,200000000,1682636477,97,3906,bien
3,It Ends with Us,25000000,334836337,131,562,moyen
4,Despicable Me 4,100000000,948140610,94,1767,moyen


**Récolte de data par scrapping du site de TMDB**

In [ ]:
# Configure le driver Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Liste des IDs de films à scraper
movie_ids = [550, 98, 59]

# Liste pour stocker les données de tous les films
all_movie_data = []

# Boucle à travers les IDs de films
for movie_id in movie_ids:
    url = f"https://www.themoviedb.org/movie/{movie_id}"
    driver.get(url)

    # Donne du temps à la page pour se charger
    time.sleep(5)

    # Initialise un dictionnaire pour stocker les données du film actuel
    movie_data = {}

    try:
        # Extrat le titre
        title = driver.find_element(By.XPATH, '//*[@id="original_header"]/div[2]/section/div[1]/h2/a').text
        movie_data['original_title'] = title.strip()
        
        # Extrait le budget
        budget = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[3]/div/div/div[2]/div/section/div[1]/div/section[1]/p[3]').text
        movie_data['budget'] = budget.strip()

        # Extrait les revenus
        revenue = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[3]/div/div/div[2]/div/section/div[1]/div/section[1]/p[4]').text
        movie_data['revenue'] = revenue.strip()

        # Extrait la durée
        runtime = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/div[2]/div/div/section/div[2]/section/div[1]/div/span[4]').text
        movie_data['runtime'] = runtime.strip()

        # Clique sur le bouton pour révéler le nombre de votes et la moyenne dans la modale
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/section/div[2]/div/div/section/div[2]/section/div[2]/div/div/div[1]/div/div[1]/div'))
        )
        button.click()
        
        # Extrait le nombre de votes et la moyenne de la modale
        vote_count = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#rating_details_window > div > div:nth-child(2) > div > div:nth-child(2) > h3'))
        ).text
        movie_data['vote_count'] = vote_count.strip()

        user_score_chart = driver.find_element(By.CLASS_NAME, 'user_score_chart')
        data_percent = user_score_chart.get_attribute('data-percent')
        movie_data['vote_average'] = data_percent.strip()

        # Ajoute les données du film actuel à la liste
        all_movie_data.append(movie_data)

    except Exception as e:
        print(f"Une erreur s'est produite pour l'ID de film {movie_id}: {e}")

# Ferme le driver
driver.quit()

# Crée un DataFrame à partir des données collectées des films
df_scrapping = pd.DataFrame(all_movie_data)

print(df_scrapping)


          original_title                   budget                    revenue  \
0             Fight Club   Budget\n$63,000,000.00  Recette \n$100,853,753.00   
1              Gladiator  Budget\n$103,000,000.00  Recette \n$465,361,176.00   
2  A History of Violence   Budget\n$32,000,000.00   Recette \n$60,740,827.00   

  runtime          vote_count vote_average  
0  2h 19m  29,148 évaluations           84  
1  2h 35m  18,270 évaluations           82  
2  1h 35m   3,167 évaluations           72  


In [ ]:
df_scrapping_copy = df_scrapping
# Nettoye la colonne 'budget'
df_scrapping_copy['budget'] = df_scrapping['budget'].str.replace('Budget\n\$', '', regex=True)
df_scrapping_copy['budget'] = df_scrapping['budget'].str.replace('[.,]', '', regex=True).astype(int)

# Nettoye la colonne 'revenue'
df_scrapping_copy['revenue'] = df_scrapping['revenue'].str.replace('Recette \n\$', '', regex=True)
df_scrapping_copy['revenue'] = df_scrapping['revenue'].str.replace('[.,]', '', regex=True).astype(int)

print(df_scrapping)

          original_title       budget      revenue runtime  \
0             Fight Club   6300000000  10085375300  2h 19m   
1              Gladiator  10300000000  46536117600  2h 35m   
2  A History of Violence   3200000000   6074082700  1h 35m   

           vote_count vote_average  
0  29,148 évaluations           84  
1  18,270 évaluations           82  
2   3,167 évaluations           72  


In [20]:
df_scrapping_copy = df_scrapping

def convert_runtime_to_minutes(runtime_str):
    hours = 0
    minutes = 0
    
    if 'h' in runtime_str:
        hours = int(runtime_str.split('h')[0].strip())
    if 'm' in runtime_str:
        minutes = int(runtime_str.split('m')[0].split()[-1].strip())
    
    total_minutes = hours * 60 + minutes
    return total_minutes

df_scrapping_copy['runtime'] = df_scrapping['runtime'].apply(convert_runtime_to_minutes)

print(df_scrapping_copy.head())

          original_title       budget      revenue  runtime  \
0             Fight Club   6300000000  10085375300      139   
1              Gladiator  10300000000  46536117600      155   
2  A History of Violence   3200000000   6074082700       95   

           vote_count vote_average  
0  29,148 évaluations           84  
1  18,270 évaluations           82  
2   3,167 évaluations           72  


In [ ]:
# Convertit la colonne 'vote_average' en numérique (int)
df_scrapping_copy['vote_average'] = pd.to_numeric(df_scrapping_copy['vote_average'])

# Convertit maintenant la colonne 'vote_average' en 'bien' ou 'moyen'
df_scrapping_copy['vote_average'] = df_scrapping_copy['vote_average'].apply(lambda x: 'bien' si x > 75 else 'moyen')

# Affiche le DataFrame mis à jour
print(df_scrapping_copy)


          original_title       budget      revenue  runtime  \
0             Fight Club   6300000000  10085375300      139   
1              Gladiator  10300000000  46536117600      155   
2  A History of Violence   3200000000   6074082700       95   

           vote_count vote_average  
0  29,148 évaluations         bien  
1  18,270 évaluations         bien  
2   3,167 évaluations        moyen  


In [ ]:
# Supprime "évaluations" et les virgules de la colonne 'vote_count'
df_scrapping_copy['vote_count'] = df_scrapping_copy['vote_count'].str.replace(' évaluations', '')
df_scrapping_copy['vote_count'] = df_scrapping_copy['vote_count'].str.replace(',', '')
print(df_scrapping_copy)

          original_title       budget      revenue  runtime vote_count  \
0             Fight Club   6300000000  10085375300      139      29148   
1              Gladiator  10300000000  46536117600      155      18270   
2  A History of Violence   3200000000   6074082700       95       3167   

  vote_average  
0         bien  
1         bien  
2        moyen  


In [ ]:
df_scrapping_copy.rename(columns={'vote_average': 'evaluation'}, inplace=True)

print(df_scrapping_copy)

          original_title       budget      revenue  runtime vote_count  \
0             Fight Club   6300000000  10085375300      139      29148   
1              Gladiator  10300000000  46536117600      155      18270   
2  A History of Violence   3200000000   6074082700       95       3167   

  evaluation  
0       bien  
1       bien  
2      moyen  


**Fusion des 5 dataframe différents pour en faire un seul**

In [ ]:
# Combinaison verticale (concaténation des DataFrames)
df_combined = pd.concat([df_mysql, df_mongo, df_api_copy, df_scrapping_copy, df_csv_copy], ignore_index=True)

print(df_combined)


                                original_title     budget     revenue runtime  \
0                                       Avatar  237000000  2787965087     162   
1     Pirates of the Caribbean: At World's End  300000000   961000000     169   
2                                      Spectre  245000000   880674609     148   
3                        The Dark Knight Rises  250000000  1084939099     165   
4                                  John Carter  260000000   284139100     132   
...                                        ...        ...         ...     ...   
4784                               El Mariachi     220000     2040920      81   
4785                                 Newlyweds       9000           0      85   
4786                 Signed, Sealed, Delivered          0           0     120   
4787                          Shanghai Calling          0           0      98   
4788                         My Date with Drew          0           0      90   

     vote_count evaluation 

In [64]:
df_combined.to_csv('combined_data.csv', index=False, encoding='utf-8')